<a href="https://colab.research.google.com/github/j54854/myColab/blob/main/optWork_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 最適化技術実験（第3部2回目）

---

## 学生番号

157xxxxx

---

## 氏名

青山　太郎

---





## 演習で利用するクラスの読込み

今回の演習課題でもJobsクラスと，前回の演習で作成したNodeクラスを利用しますので，最初にそれらを読み込んでおきましょう．まずはJobsクラスからです．





In [ ]:
import csv
import random
import sys
import time
import plotly.figure_factory as ff

class Jobs:
    def __init__(self, pt=None):
        if pt:  # pt must be 2-dimensional list of integers
            self.pt = pt  # pt[j][m]: processing time of job j on machine m
            self.J = len(self.pt)  # number of jobs
            self.M = len(self.pt[0])  # number of machines
        else:  # if pt is not given
            pass  # do nothing (pt, J, M sould be set later)

    def save_in_csv_file(self, filename):  # pt_jm is saved in a csv file
        with open(filename, 'w') as f:
            writer = csv.writer(f)
            for j in range(self.J):
                writer.writerow(self.pt[j])

    def set_by_csv_file(self, filename):  # pt, J & M are set by a csv file, where rows correespond to jobs and columns correspond to machines
        with open(filename) as f:
            reader = csv.reader(f)
            self.pt = [
                [int(pt) for pt in row] for row in reader
            ]
            self.J = len(self.pt)
            self.M = len(self.pt[0])

    def set_randomly(self, J, M):  # pt_jm is set randomly for specified J & M
        self.J = J
        self.M = M
        self.pt = [
            [random.randint(1, 60) for m in range(M)] for j in range(J)
        ]

    def make_schedule(self, seq=None):
        if not seq:  # if seq is not given, it is set randomly
            seq = list(range(self.J))  # seq = [0, 1, 2, ..., J]
            random.shuffle(seq)  # randomize the order of elements in seq
        self.st = [[0] *self.M for j in range(self.J)]  # start time of job j on machine m is initilized to be 0
        last_ct = [0 for m in range(self.M)]  # when each machine finishes the last job (& becomes available) is also set to be 0
        for j in seq:
            last_ct_j = 0  # when job j is completed on the last machine (& becomes ready) is initially set to be 0
            for m in range(self.M):
                self.st[j][m] = max(last_ct_j, last_ct[m])
                last_ct[m] = self.st[j][m] +self.pt[j][m]
                last_ct_j = self.st[j][m] +self.pt[j][m]
        print('Job sequence = ', seq)
        print('Makespan = {}'.format(last_ct_j))

    def draw_schedule(self):
        operations = []
        for j in range(self.J):
            for m in range(self.M):
                day = '2020-12-01 '  # dummy date
                st_h, st_m = divmod(self.st[j][m], 60)
                ct_h, ct_m = divmod(self.st[j][m] +self.pt[j][m], 60)
                st_daytime = day +'{:02}:{:02}'.format(st_h, st_m)
                ct_daytime = day +'{:02}:{:02}'.format(ct_h, ct_m)
                operations.append(
                    dict(Task='Machine'+str(m), Start=st_daytime, Finish=ct_daytime, Resource='Job'+str(j))
                )
        fig = ff.create_gantt(operations, index_col='Resource', show_colorbar=True, group_tasks=True)
        fig.show()

次にNodeクラスを読み込みます．なお，下記のサンプルコードは各メソッドの中身が記述されていないスケルトンですので，前回の演習課題で作成したコードに置き換えてから実行するようにします．

In [ ]:
class Node:
    def __init__(self, jobs, seq=None):
        self.jobs = jobs  # all jobs (Jobs class instance)
        self.seq = []  # (partial) job sequence
        self.rest = list(range(jobs.J))  # unassigned jobs
        self.last_ct = [0] *jobs.M  # when each machine finishes the last job (& becomes available) is initially set to be 0
        if seq:
            self.set_seq(seq)

    @property
    def makespan(self):
        pass

    def assign(self, j):  # assign job j in rest
        pass

    def set_seq(self, seq):  # set (partial) job sequence at a time
        pass

    def is_complete(self):  # is this a full schedule?
        pass

## ノードの集合を表すNodeSetクラスの導入

探索木は，まだジョブの順序が全く指定されていない根ノード（可能なすべての順列からなる集合）から始めて，それに分枝操作を適用して子ノードに分割していく（互いに素な順列の部分集合に分割していく）ことで構成していきます．ここに，分枝操作とは，部分スケジュールにジョブを1つずつ割り付けていくことで，ジョブの順序を前から順に徐々に確定させていくことに対応しています．

この探索木を構成するプロセスを，前回は，関数の再帰呼出しで実装しましたが，今回は，別の方法で実装してみることにします．すなわち，まだ分枝操作を適用していない未分枝ノードの集合を保持しておき，毎回，その集合から１つ選んでそのノードに分枝操作を適用するという流れで探索木を構成していきます．この方法を用いると，ノードをチェックしていく順序を変更することが可能になります．

このために，まず，保持しておく未分枝ノードの集合を表現するためのNodeSetクラスを導入しましょう．

下のサンプルコードの`__init__()`メソッドを見てください．ノード（Nodeクラスのインスタンス）の集合をnodesという名称のリストに保持するようになっており，そのリストが根ノードのみを含むように初期化されていることがわかります．また，best_seqとbest_msには，探索によって既にチェックしたものの中で最も好ましい順列とその順列に対応するスケジュールのメイクスパンを格納するようになっています（sys.maxsizeは非常に大きな整数値と考えておけばOKです）．

その下にいくつかのメソッドのスケルトンが用意されていますが，予想通り，それらは以下の演習課題で利用することになります．

In [ ]:
class NodeSet:  # the set of nodes to be checked
    def __init__(self, jobs):
        self.nodes = [Node(jobs)]  # only the root node
        self.best_seq = None  # best job sequence found so far
        self.best_ms = sys.maxsize  # its makespan

    def enumerate_all(self):
        while self.nodes:
            node = self.nodes.pop(0)
            # print(node.seq)
            pass

    def enumerate_all_d(self):
        pass

    def search_all(self):
        pass

    def search_bab(self):
        pass

## 演習課題1: NodeSetクラスを用いた順列の列挙

最初に，関数の再帰呼出しではなく，未分枝ノードの集合を用いて順列を列挙していくことに挑戦してみましょう．

### 課題1-1

上のサンプルコード内のenumerate_all()メソッドを完成させてみてください．whileループとその中で実行する最初の処理までが用意されています．これは，未分枝ノードの集合を表すリストself.nodesが空でない限りループを繰り返し，そのループの中では，まずself.nodesの先頭の要素を取り出してそれをnodeで参照できるようにしています．

その下のpassを削除して必要なコードを追加して順列を列挙できるようにしましょう．下記のコードを実行して可能なすべての順列が書き出されれば完成です．

In [ ]:
pt = [
        [35, 7, 18, 20],
        [4, 60, 12, 10],
        [15, 15, 20, 30],
        [11, 25, 10, 35],
        [33, 1, 45, 12]
    ]
jobs = Jobs(pt)

tree = NodeSet(jobs)
tree.enumerate_all()

コメントアウトされている3行目のprint文を復活させると，self.nodesからnodeが取り出されるたびに，それが葉ノードであるかどうかに関わらず，node.seqが書き出されるようになります．これで，探索木のノードがどのような順序でチェックされていったかを確認してみましょう．普通に実装した場合，上記のenumerate_all()メソッドでは，この順序は幅優先になっているはずです．

前回の演習課題2で実装した再帰呼出しによる順列列挙関数についても，同じようにしてノードがチェックされていく順序を確認してみてください．そちらは深さ優先になっているはずです．

関数の再帰呼出しではなく，今回のように未分岐ノードの集合を用いて順列を列挙していくようにすると，少しの工夫で，ノードをチェックしていく順序を変更することができます．これを次の演習課題で確認してみましょう．

## 課題1-2

関数の再帰呼出しを用いた場合と同じ順序でノードがチェックされていくように，enumerate_all()メソッドに微修正を加えてみよう．解答のコードは，enumerate_all_d()メソッドの中に書き入れてください．仕上がったら下のコードで結果を確認してみること．


In [ ]:
tree = NodeSet(jobs)
tree.enumerate_all_d()

## 演習課題2: NodeSetクラスを用いたスケジュールの全数探索

続いて，上と同じノード集合を利用して，ジョブの順列（に対応するスケジュール）の全数探索を行ってみましょう．メイクスパンを最小にするスケジュール（に対応する順列）を探します．

### 課題2-1

スケジュールの全数探索を実行できるように上のenumerate_all()メソッドを拡張し，サンプルコード内のsearch_all()メソッドを完成させてください．

実装し終わったら，下記のコードで動作を確認しておきましょう．最後に最適なスケジュールが描画されれば完成です．


In [ ]:
tree = NodeSet(jobs)
tree.search_all()

## 演習課題3: 簡単な分枝限定法への拡張

探索木の各ノードはNodeクラスのインスタンスになっています．ここで，Nodeクラスの定義を思い出しましょう．各ノード（nodeと表記します）では，node.seqに含まれるジョブはそれに指定された順で既に部分スケジュールに割り付けられていました．そして，その部分スケジュールのメイクスパンの値はnode.makespanで参照できるようになっていました．

このとき，nodeを起点として分枝操作を繰り返して作られる任意の葉ノード（のもつ完全な順列）に対応するスケジュールのメイクスパンは，このnode.makespanの値未満になることはないということが保証されます．この意味で，node.makespanの値は，node（で規定される順列の部分集合）に含まれるスケジュールのメイクスパンの下界（必ず最小値以下になる値）になっています．

このように，各ノードの評価値として，そこから派生して作られるスケジュールのメイクスパンの下界値が得られる場合，それを用いて，探索木の枝刈りを行うことができます．すなわち，探索の過程で既にある葉ノード（に対応するスケジュール，すなわち暫定解）が見つかっていて，そのメイクスパンの値を評価済みだとします．このとき，その暫定解のメイクスパンの値よりも下界値が大きいノードからは，暫定解よりもメイクスパンの短いスケジュールを作ることはできないということが保証されますので，そのノードはそれ以上分枝して調べる必要はないということになります．すなわち，そのノードから先の枝は刈り取ってしまうことができるわけです．この処理を限定操作と呼びます．

分枝操作に限定操作を組み合わせることで，最適なスケジュールを見つける探索のプロセスを効率化することができます．これを分枝限定法（Branch and Bound Method: BAB）と呼びます．また，上記の各ノードの評価値は，次にどの未分枝ノードをチェックするかの順序付けにも利用できます．例えば，下界の値が最も小さいノードが有望だと考えて，そのノードから順にチェックしていく方法を最良下界探索と呼びます．

ここでは，この分枝限定法と最良下界探索の考え方を取り入れて，上の全数探索のメソッドを拡張してみましょう．

### 課題3-1

上で作成したsearch_all()メソッドに分枝限定法と最良下界探索の考え方を導入して，search_bab()メソッドに拡張してみましょう．各ノード（node）の下界としては，node.makespanの値を利用すれば十分です（が，余裕があれば，それをより良い下界に置きかえることにも挑戦してみてください）．

実装し終わったら，下記のコードで動作を確認しておきましょう．最後に最適なスケジュールが描画されれば完成です．


In [ ]:
tree = NodeSet(jobs)
tree.search_bab()

## 考察課題4

最後に，今回の講義と課題に関連して，下記の項目について考察してみましょう．下の欄に直接書き込んで提出してください．

- 分枝限定法で利用している各ノードの評価値は，そのノード（すなわち，順列の部分集合）に含まれるスケジュールのメイクスパンの下界（必ず最小値以下になる値）になっていました．この下界の精度を高める（すなわち，最小値に近づける）と枝刈りの効率は向上します．下界の精度を高める方法について考えてみてください（ただし，精度を高めると，一般に，下界の計算にかかる時間は長くなってしまう傾向がありますので，必ずしも高精度にすることが望ましいとは限りません）．

- 探索木のノードをチェックしていく順序の決め方として，深さ優先探索，幅優先探索，最良下界探索の3つの方法をみてきました．これら以外に，順序を決めるのに有効な方法はないでしょうか．

- 分枝限定法で，各ノードの評価値として，メイクスパンの厳密な下界ではなく，必ずしも下界であるとは限らない（メイクスパンの最小値よりも大きな値をとる可能性が残る）メイクスパンの推定値を用いるとどうなるでしょうか．

### 課題4-1

この文を消して，ここに，自分の考察を記入してください．



## まとめ

演習課題のコードは，所定の箇所に（passを消してから）直接書き込んでください．提出前に必ず動作を確認しておくこと．動作しないコードは採点対象になりません．また，考察課題も，所定の欄に直接書き込んでもらえればOKです．

最後に，このGoogle Colabのノートブックを

★☆★ <b>ipynb形式</b> ★☆★

で保存したファイルをコースパワーから提出すること．提出の〆切は，

★☆★ <b>18:20</b> ★☆★

とします．